# 🧹 Tratamento de Dados de Combustíveis - 2004

Este notebook tem como objetivo realizar o tratamento inicial dos dados de preços de combustíveis da ANP para o ano de 2004. Aqui, vamos garantir que os dados estejam limpos e estruturados para análises futuras.

## Objetivos:
- Carregar os dados dos dois semestres de 2004
- Verificar a consistência das colunas
- Identificar e tratar valores ausentes
- Preparar os dados para análises posteriores

In [32]:
import pandas as pd
import os

# Caminhos dos arquivos
caminho_1s = 'dados_anp_ca/ca-2004-01.csv'
caminho_2s = 'dados_anp_ca/ca-2004-02.csv'

def exibir_colunas(df, nome):
    print(f"\n📋 Colunas do {nome}:")
    print("-" * 50)
    for i, col in enumerate(df.columns, 1):
        tipo = str(df[col].dtype)
        print(f"{i:2d}. {col:<40} ({tipo})")
    print(f"\nTotal: {len(df.columns)} colunas")

# Lendo os arquivos CSV
try:
    # Lendo o primeiro semestre
    print("📂 Carregando dados do primeiro semestre...")
    df_1s = pd.read_csv(caminho_1s, encoding='utf-8', sep=';', decimal=',')
    print(f"✅ Registros carregados: {len(df_1s):,}")
    exibir_colunas(df_1s, "Primeiro Semestre")
    
    # Lendo o segundo semestre
    print("\n📂 Carregando dados do segundo semestre...")
    df_2s = pd.read_csv(caminho_2s, encoding='utf-8', sep=';', decimal=',')
    print(f"✅ Registros carregados: {len(df_2s):,}")
    exibir_colunas(df_2s, "Segundo Semestre")
    
    # Verificação de colunas iguais
    colunas_1s = set(df_1s.columns)
    colunas_2s = set(df_2s.columns)
    
    if colunas_1s == colunas_2s:
        print("\n✅ Todas as colunas são iguais entre os dois semestres.")
    else:
        print("\n❌ As colunas são diferentes entre os semestres.")
        
        # Mostrar diferenças
        apenas_1s = colunas_1s - colunas_2s
        apenas_2s = colunas_2s - colunas_1s
        
        if apenas_1s:
            print(f"\n🔍 Colunas apenas no 1º semestre ({len(apenas_1s)}):")
            for col in sorted(apenas_1s):
                print(f"   - {col}")
                
        if apenas_2s:
            print(f"\n🔍 Colunas apenas no 2º semestre ({len(apenas_2s)}):")
            for col in sorted(apenas_2s):
                print(f"   - {col}")
    
    # Armazenando os DataFrames
    dados_2004 = {
        'primeiro_semestre': df_1s,
        'segundo_semestre': df_2s
    }
    
except UnicodeDecodeError:
    print("\n❌ Erro de codificação: O arquivo não está em UTF-8. Tentando com ISO-8859-1...")
    try:
        # Tentar novamente com ISO-8859-1 se UTF-8 falhar
        df_1s = pd.read_csv(caminho_1s, encoding='ISO-8859-1', sep=';', decimal=',')
        df_2s = pd.read_csv(caminho_2s, encoding='ISO-8859-1', sep=';', decimal=',')
        print("✅ Arquivos carregados com sucesso usando ISO-8859-1")
        
        # Exibir informações novamente
        exibir_colunas(df_1s, "Primeiro Semestre (ISO-8859-1)")
        exibir_colunas(df_2s, "Segundo Semestre (ISO-8859-1)")
        
        # Armazenar os DataFrames
        dados_2004 = {
            'primeiro_semestre': df_1s,
            'segundo_semestre': df_2s
        }
        
    except Exception as e:
        print(f"\n❌ Erro ao carregar os arquivos: {str(e)}")
        
except Exception as e:
    print(f"\n❌ Erro ao processar os arquivos: {str(e)}")

📂 Carregando dados do primeiro semestre...
✅ Registros carregados: 281,531

📋 Colunas do Primeiro Semestre:
--------------------------------------------------
 1. Regiao - Sigla                           (object)
 2. Estado - Sigla                           (object)
 3. Municipio                                (object)
 4. Revenda                                  (object)
 5. CNPJ da Revenda                          (object)
 6. Nome da Rua                              (object)
 7. Numero Rua                               (object)
 8. Complemento                              (object)
 9. Bairro                                   (object)
10. Cep                                      (object)
11. Produto                                  (object)
12. Data da Coleta                           (object)
13. Valor de Venda                           (float64)
14. Valor de Compra                          (float64)
15. Unidade de Medida                        (object)
16. Bandeira                 

In [33]:
def analisar_colunas(df, nome_periodo):
    print(f"\n🔍 Análise de Colunas - {nome_periodo}")
    print("="*70)
    
    # Criar DataFrame com métricas de cada coluna
    analise = pd.DataFrame({
        'Tipo': df.dtypes,
        'Valores_Únicos': df.nunique(),
        'Valores_Nulos': df.isnull().sum(),
        '%_Nulos': (df.isnull().sum() / len(df) * 100).round(2)
    })
    
    # Classificar colunas
    def classificar_coluna(linha):
        # Colunas com alta cardinalidade ou muitos nulos são menos úteis
        if linha['%_Nulos'] > 50:
            return '❌ Baixa Utilidade'
        elif linha['Valores_Únicos'] == len(df):
            return '⚠️ Identificador Único (CNPJ, ID, etc)'
        elif linha['Tipo'] == 'object':
            if linha['Valores_Únicos'] < 50:
                return '✅ Categórica'
            else:
                return '📝 Texto Livre'
        elif 'data' in linha.name.lower():
            return '📅 Data (converter para datetime)'
        elif 'valor' in linha.name.lower() or 'preço' in linha.name.lower():
            return '💰 Métrica Numérica (importante)'
        else:
            return '🔢 Numérica'
    
    analise['Classificação'] = analise.apply(classificar_coluna, axis=1)
    
    # Ordenar por classificação e nome
    analise = analise.sort_values(['Classificação', 'Valores_Únicos'])
    
    # Exibir resultados
    with pd.option_context('display.max_rows', None, 'display.width', 1000):
        display(analise)
    
    return analise

# Analisar colunas de cada período
print("🔍 INICIANDO ANÁLISE DAS COLUNAS")
analise_1s = analisar_colunas(df_1s, "Primeiro Semestre")
analise_2s = analisar_colunas(df_2s, "Segundo Semestre")

# Identificar diferenças entre os períodos
colunas_diferentes = set(df_1s.columns).symmetric_difference(df_2s.columns)
if colunas_diferentes:
    print("\n⚠️  ATENÇÃO: Diferenças nas colunas entre os semestres:", 
          ', '.join(colunas_diferentes))

🔍 INICIANDO ANÁLISE DAS COLUNAS

🔍 Análise de Colunas - Primeiro Semestre


,Tipo,Valores_Únicos,Valores_Nulos,%_Nulos,Classificação
Unidade de Medida,object,2,0,0.00,✅ Categórica
Produto,object,4,0,0.00,✅ Categórica
Regiao - Sigla,object,5,0,0.00,✅ Categórica
Estado - Sigla,object,27,0,0.00,✅ Categórica
Data da Coleta,object,33,0,0.00,✅ Categórica
Complemento,object,2216,189298,67.24,❌ Baixa Utilidade
Valor de Venda,float64,1465,0,0.00,💰 Métrica Numérica (importante)
Valor de Compra,float64,18224,86296,30.65,💰 Métrica Numérica (importante)
Bandeira,object,104,0,0.00,📝 Texto Livre
Municipio,object,616,0,0.00,📝 Texto Livre



🔍 Análise de Colunas - Segundo Semestre


,Tipo,Valores_Únicos,Valores_Nulos,%_Nulos,Classificação
Unidade de Medida,object,2,0,0.00,✅ Categórica
Produto,object,4,0,0.00,✅ Categórica
Regiao - Sigla,object,5,0,0.00,✅ Categórica
Estado - Sigla,object,27,0,0.00,✅ Categórica
Complemento,object,2240,616266,67.33,❌ Baixa Utilidade
Valor de Venda,float64,1772,0,0.00,💰 Métrica Numérica (importante)
Valor de Compra,float64,30403,288708,31.54,💰 Métrica Numérica (importante)
Bandeira,object,106,0,0.00,📝 Texto Livre
Data da Coleta,object,110,0,0.00,📝 Texto Livre
Municipio,object,616,0,0.00,📝 Texto Livre


# 🔍 Seleção Estratégica de Colunas para Análise

## 📊 Colunas Selecionadas e Sua Relevância

### 📍 Dados Geográficos
- **Região**: Permite análises macro por região do país
- **Estado**: Essencial para análises estaduais e comparações regionais
- **Município**: Para análises detalhadas em nível municipal
- **Bandeira**: Importante para análise de mercado e comparação entre postos

### 🏷️ Dados de Identificação
- **Revenda**: Identifica o estabelecimento vendedor
- **CNPJ da Revenda**: Identificador único para análise por estabelecimento
- **Produto**: Tipo de combustível (gasolina, diesel, etanol, etc)

### 📅 Dados Temporais
- **Data da Coleta**: Fundamental para análise de séries temporais, tendências e sazonalidade

### 💰 Dados Financeiros
- **Valor de Venda**: Preço final ao consumidor
- **Valor de Compra**: Custo para a revenda
- **Unidade de Medida**: Garante que as comparações sejam feitas corretamente

### 🗑️ Colunas Removidas
As demais colunas foram removidas por:
- Conterem informações redundantes
- Não serem relevantes para as análises propostas
- Estarem vazias ou com baixa taxa de preenchimento
- Não adicionarem valor analítico significativo

## 🎯 Objetivo
Esta seleção visa otimizar o conjunto de dados para análises de preços de combustíveis, mantendo apenas as informações mais relevantes para:
- Análises de preços por região/estado/município
- Comparação entre bandeiras de postos
- Análises temporais de variação de preços
- Cálculo de margens de lucro (diferença entre venda e compra)

In [34]:
# Dicionário de padronização dos nomes das colunas
padrao_colunas = {
    'Regiao - Sigla': 'regiao_sigla',
    'Estado - Sigla': 'estado_sigla',
    'Municipio': 'municipio',
    'Revenda': 'revenda',
    'CNPJ da Revenda': 'cnpj_revenda',
    'Produto': 'produto',
    'Data da Coleta': 'data_coleta',
    'Valor de Venda': 'valor_venda',
    'Valor de Compra': 'valor_compra',
    'Unidade de Medida': 'unidade_medida',
    'Bandeira': 'bandeira'
}

# Lista das colunas que serão mantidas (usando os nomes originais)
colunas_essenciais = [
    'Regiao - Sigla',
    'Estado - Sigla',
    'Municipio',
    'Revenda',
    'CNPJ da Revenda',
    'Produto',
    'Data da Coleta',
    'Valor de Venda',
    'Valor de Compra',
    'Unidade de Medida',
    'Bandeira'
]

# Filtrar e renomear as colunas
def preparar_dataframe(df):
    # Filtrar colunas
    df = df[colunas_essenciais].copy()
    # Renomear colunas
    return df.rename(columns=padrao_colunas)

# Aplicar a ambos os dataframes
df_1s = preparar_dataframe(df_1s)
df_2s = preparar_dataframe(df_2s)

# Verificar o resultado
print("🔍 Colunas após a filtragem e padronização:")
print("-" * 50)
print(df_1s.columns.tolist())

# Mostrar as primeiras linhas do primeiro semestre como exemplo
print("\n📋 Dados do Primeiro Semestre (amostra):")
display(df_1s.head(2))

# Mostrar as primeiras linhas do segundo semestre como exemplo
print("\n📋 Dados do Segundo Semestre (amostra):")
display(df_2s.head(2))

# Converter a coluna de data
df_1s['data_coleta'] = pd.to_datetime(df_1s['data_coleta'], dayfirst=True)
df_2s['data_coleta'] = pd.to_datetime(df_2s['data_coleta'], dayfirst=True)

print("\n✅ Dados preparados com sucesso!")
print(f"Primeiro semestre: {len(df_1s)} registros")
print(f"Segundo semestre: {len(df_2s)} registros")

🔍 Colunas após a filtragem e padronização:
--------------------------------------------------
['regiao_sigla', 'estado_sigla', 'municipio', 'revenda', 'cnpj_revenda', 'produto', 'data_coleta', 'valor_venda', 'valor_compra', 'unidade_medida', 'bandeira']

📋 Dados do Primeiro Semestre (amostra):


,regiao_sigla,estado_sigla,municipio,revenda,cnpj_revenda,produto,data_coleta,valor_venda,valor_compra,unidade_medida,bandeira
0,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,GASOLINA,11/05/2004,1.967,1.6623,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
1,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,ETANOL,11/05/2004,0.899,0.6282,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.



📋 Dados do Segundo Semestre (amostra):


,regiao_sigla,estado_sigla,municipio,revenda,cnpj_revenda,produto,data_coleta,valor_venda,valor_compra,unidade_medida,bandeira
0,CO,MS,CAMPO GRANDE,AUTO POSTO APARECIDA DO NORTE LTDA,86.807.609/0001-92,GASOLINA,01/07/2004,2.71,2.0888,R$ / litro,LIQUIGÁS
1,CO,MS,CAMPO GRANDE,AUTO POSTO APARECIDA DO NORTE LTDA,86.807.609/0001-92,ETANOL,01/07/2004,1.71,1.1877,R$ / litro,LIQUIGÁS



✅ Dados preparados com sucesso!
Primeiro semestre: 281531 registros
Segundo semestre: 915317 registros


# 🔍 Análise de Valores Nulos

## 📊 Visão Geral dos Dados Ausentes

Vamos analisar a quantidade e porcentagem de valores nulos em cada coluna do nosso conjunto de dados. Esta análise é crucial para entendermos a qualidade dos dados e decidirmos as melhores estratégias de tratamento.

### Por que essa análise é importante?
1. **Qualidade dos Dados**: Identificar colunas com muitos valores ausentes que podem comprometer as análises.
2. **Tomada de Decisão**: Decidir se devemos preencher, remover ou manter os valores nulos.
3. **Impacto nas Análises**: Entender como os valores ausentes podem afetar os resultados das nossas análises.

### Próximos Passos:
1. Verificar a quantidade de valores nulos por coluna
2. Calcular a porcentagem de valores nulos em relação ao total de registros
3. Identificar padrões nos dados ausentes
4. Decidir a melhor estratégia de tratamento para cada caso

Vamos começar executando a análise de valores nulos para cada semestre separadamente.

In [35]:
# Função para análise de valores nulos
def analisar_nulos(df, nome_periodo):
    print(f"\n🔍 Análise de Valores Nulos - {nome_periodo}")
    print("="*70)
    
    # Calcular totais
    total_registros = len(df)
    print(f"📊 Total de registros: {total_registros:,}")
    
    # Criar DataFrame com análise de nulos
    nulos = pd.DataFrame({
        'Valores_Nulos': df.isnull().sum(),
        'Percentual_Nulos': (df.isnull().sum() / total_registros * 100).round(2)
    }).sort_values('Valores_Nulos', ascending=False)
    
    # Exibir apenas colunas com valores nulos
    nulos = nulos[nulos['Valores_Nulos'] > 0]
    
    if len(nulos) == 0:
        print("✅ Nenhum valor nulo encontrado!")
    else:
        print(f"⚠️  Colunas com valores nulos: {len(nulos)} de {len(df.columns)} colunas")
        display(nulos)
    
    return nulos

# Executar análise para ambos os semestres
nulos_1s = analisar_nulos(df_1s, "Primeiro Semestre")
nulos_2s = analisar_nulos(df_2s, "Segundo Semestre")


🔍 Análise de Valores Nulos - Primeiro Semestre
📊 Total de registros: 281,531
⚠️  Colunas com valores nulos: 1 de 11 colunas


,Valores_Nulos,Percentual_Nulos
valor_compra,86296,30.65



🔍 Análise de Valores Nulos - Segundo Semestre
📊 Total de registros: 915,317
⚠️  Colunas com valores nulos: 1 de 11 colunas


,Valores_Nulos,Percentual_Nulos
valor_compra,288708,31.54


# 🧹 Tratamento de Valores Ausentes

## 📊 Estratégia de Preenchimento

### Para Valores Numéricos (ex: Valor de Compra):
- **Método**: Preenchimento com a mediana do grupo
- **Vantagens**:
  - Menos sensível a outliers que a média
  - Mantém a distribuição original dos dados
  - Considera o contexto (produto, bandeira, região)

### Para Valores Categóricos (ex: Bandeira, Município):
- **Método**: Preenchimento com categoria "Desconhecido"
- **Vantagens**:
  - Não inventa dados que não existem
  - Mantém a transparência do tratamento
  - Permite identificar registros que precisam de atenção especial

In [36]:
# Para o primeiro semestre
print("\n" + "="*60)
print("📊 TRATAMENTO - PRIMEIRO SEMESTRE".center(60))
print("="*60)

# Tratar valores numéricos
colunas_agrupar = ['produto', 'estado_sigla', 'municipio']
df_1s = imputar_valores_numericos(df_1s, 'valor_venda', colunas_agrupar)
df_1s = imputar_valores_numericos(df_1s, 'valor_compra', colunas_agrupar)

# Tratar valores categóricos
colunas_cat = ['bandeira', 'unidade_medida', 'regiao_sigla']
df_1s = imputar_valores_categoricos(df_1s, colunas_cat)

# Para o segundo semestre
print("\n" + "="*60)
print("📊 TRATAMENTO - SEGUNDO SEMESTRE".center(60))
print("="*60)

# Tratar valores numéricos
df_2s = imputar_valores_numericos(df_2s, 'valor_venda', colunas_agrupar)
df_2s = imputar_valores_numericos(df_2s, 'valor_compra', colunas_agrupar)

# Tratar valores categóricos
df_2s = imputar_valores_categoricos(df_2s, colunas_cat)

print("\n✅ Tratamento concluído para ambos os semestres!")
print(f"- Primeiro semestre: {len(df_1s)} registros")
print(f"- Segundo semestre: {len(df_2s)} registros")


              📊 TRATAMENTO - PRIMEIRO SEMESTRE              

🔧 Iniciando tratamento para a coluna 'valor_venda'
------------------------------------------------------------
Valores nulos antes do tratamento: 0 (0.00%)
Valores nulos após o tratamento: 0
✅ Tratamento concluído com sucesso!

🔧 Iniciando tratamento para a coluna 'valor_compra'
------------------------------------------------------------
Valores nulos antes do tratamento: 86,296 (30.65%)
Valores nulos após o tratamento: 0
✅ Tratamento concluído com sucesso!

🔧 Iniciando tratamento para colunas categóricas
------------------------------------------------------------
✅ Tratamento de colunas categóricas concluído!

              📊 TRATAMENTO - SEGUNDO SEMESTRE               

🔧 Iniciando tratamento para a coluna 'valor_venda'
------------------------------------------------------------
Valores nulos antes do tratamento: 0 (0.00%)
Valores nulos após o tratamento: 0
✅ Tratamento concluído com sucesso!

🔧 Iniciando tratamento p

In [37]:
# Verificar se ainda existem valores nulos
print("\n" + "="*70)
print("VERIFICAÇÃO FINAL DE VALORES NULOS")
print("="*70)
print("\nPrimeiro Semestre:")
display(df_1s.isnull().sum())
print("\nSegundo Semestre:")
display(df_2s.isnull().sum())


VERIFICAÇÃO FINAL DE VALORES NULOS

Primeiro Semestre:


regiao_sigla      0
estado_sigla      0
municipio         0
revenda           0
cnpj_revenda      0
produto           0
data_coleta       0
valor_venda       0
valor_compra      0
unidade_medida    0
bandeira          0
dtype: int64


Segundo Semestre:


regiao_sigla      0
estado_sigla      0
municipio         0
revenda           0
cnpj_revenda      0
produto           0
data_coleta       0
valor_venda       0
valor_compra      0
unidade_medida    0
bandeira          0
dtype: int64

# 🔍 Análise de Inconsistências nos Preços

## 📌 Objetivo
Identificar e corrigir registros onde `Valor de Venda < Valor de Compra`, o que indica prejuízo financeiro.

## 💡 Por que analisar?
- **Erros de Dados**: Identificar registros com valores incorretos
- **Saúde Financeira**: Garantir margens positivas
- **Qualidade**: Manter a confiabilidade do dataset

## 📊 O que vamos verificar?
- Quantos registros estão com prejuízo
- Quais produtos/bandeiras têm mais problemas
- Qual o impacto financeiro dessas inconsistências

## 🔧 Próximos Passos
1. Identificar registros problemáticos
2. Analisar padrões
3. Corrigir os valores
4. Validar os resultados

In [38]:
def verificar_inconsistencias_precos(df, nome_periodo):
    """
    Verifica se existem valores de venda menores que os de compra,
    o que seria uma inconsistência nos dados.
    
    Parâmetros:
    - df: DataFrame com os dados
    - nome_periodo: Nome do período para exibição
    
    Retorna:
    - DataFrame com as inconsistências encontradas ou None se não houver
    """
    print(f"\n🔍 Verificando Inconsistências de Preços - {nome_periodo}")
    print("="*70)
    
    # Verificar se existem valores de venda menores que compra
    inconsistencias = df[df['valor_venda'] < df['valor_compra']].copy()
    total_inconsistencias = len(inconsistencias)
    total_linhas = len(df)
    percentual = (total_inconsistencias / total_linhas) * 100 if total_linhas > 0 else 0
    
    # Resumo Executivo
    print("\n" + "="*70)
    print("📊 RESUMO DAS INCONSISTÊNCIAS")
    print("="*70)
    print(f"✅ Total de registros analisados: {total_linhas:,}")
    print(f"❌ Registros com VENDA < COMPRA: {total_inconsistencias:,} ({percentual:.2f}%)")
    print("="*70)
    
    if total_inconsistencias > 0:
        # Calcular a diferença
        inconsistencias['diferenca'] = inconsistencias['valor_compra'] - inconsistencias['valor_venda']
        impacto_total = inconsistencias['diferenca'].sum()
        
        print("\n📋 DETALHES DAS INCONSISTÊNCIAS:")
        print(f"  - Diferença Média: R$ {inconsistencias['diferenca'].mean():.2f}")
        print(f"  - Maior Diferença: R$ {inconsistencias['diferenca'].max():.2f}")
        print(f"  - Menor Diferença: R$ {inconsistencias['diferenca'].min():.2f}")
        print(f"  - Impacto Total:  R$ {impacto_total:,.2f}")
        
        print("\n📊 AMOSTRA DAS INCONSISTÊNCIAS (Top 5):")
        display(inconsistencias[['produto', 'bandeira', 'valor_compra', 
                              'valor_venda', 'diferenca']]
               .sort_values('diferenca', ascending=False).head())
        
        # Análise por produto
        print("\n📈 ANÁLISE POR PRODUTO:")
        analise_produto = inconsistencias.groupby('produto').agg(
            qtd_inconsistencias=('produto', 'size'),
            media_diferenca=('diferenca', 'mean'),
            impacto_total=('diferenca', 'sum')
        ).sort_values('qtd_inconsistencias', ascending=False)
        display(analise_produto)
        
        return inconsistencias
    else:
        print("\n✅ Nenhuma inconsistência encontrada!")
        return None

# Executar verificação para ambos os semestres
print("="*70)
print("ANÁLISE DE INCONSISTÊNCIAS NOS PREÇOS (venda < compra)")
print("="*70)

# Verificar primeiro semestre
inconsistencias_1s = verificar_inconsistencias_precos(df_1s, "Primeiro Semestre")

# Verificar segundo semestre
inconsistencias_2s = verificar_inconsistencias_precos(df_2s, "Segundo Semestre")

ANÁLISE DE INCONSISTÊNCIAS NOS PREÇOS (venda < compra)

🔍 Verificando Inconsistências de Preços - Primeiro Semestre

📊 RESUMO DAS INCONSISTÊNCIAS
✅ Total de registros analisados: 281,531
❌ Registros com VENDA < COMPRA: 1,586 (0.56%)

📋 DETALHES DAS INCONSISTÊNCIAS:
  - Diferença Média: R$ 0.05
  - Maior Diferença: R$ 0.65
  - Menor Diferença: R$ 0.00
  - Impacto Total:  R$ 84.95

📊 AMOSTRA DAS INCONSISTÊNCIAS (Top 5):


,produto,bandeira,valor_compra,valor_venda,diferenca
232136,GASOLINA,RAIZEN,2.7345,2.089,0.6455
232731,GASOLINA,RAIZEN,2.7345,2.099,0.6355
18094,ETANOL,RAIZEN,1.4123,0.788,0.6243
88410,ETANOL,RAIZEN,1.4120,0.788,0.6240
75889,ETANOL,BRANCA,1.4143,0.949,0.4653



📈 ANÁLISE POR PRODUTO:


,qtd_inconsistencias,media_diferenca,impacto_total
produto,,,
ETANOL,781,0.051490,40.21369
GASOLINA,673,0.045477,30.60630
GNV,104,0.129273,13.44440
DIESEL,28,0.024566,0.68785



🔍 Verificando Inconsistências de Preços - Segundo Semestre

📊 RESUMO DAS INCONSISTÊNCIAS
✅ Total de registros analisados: 915,317
❌ Registros com VENDA < COMPRA: 10,123 (1.11%)

📋 DETALHES DAS INCONSISTÊNCIAS:
  - Diferença Média: R$ 0.06
  - Maior Diferença: R$ 0.49
  - Menor Diferença: R$ 0.00
  - Impacto Total:  R$ 625.32

📊 AMOSTRA DAS INCONSISTÊNCIAS (Top 5):


,produto,bandeira,valor_compra,valor_venda,diferenca
464505,ETANOL,BRANCA,1.6805,1.188,0.4925
33889,GASOLINA,BRANCA,2.4730,1.999,0.4740
499729,ETANOL,BRANCA,1.6805,1.239,0.4415
534957,ETANOL,BRANCA,1.6805,1.239,0.4415
41270,ETANOL,BRANCA,1.4200,0.990,0.4300



📈 ANÁLISE POR PRODUTO:


,qtd_inconsistencias,media_diferenca,impacto_total
produto,,,
ETANOL,7912,0.063471,502.180545
GASOLINA,1839,0.032517,59.799020
GNV,248,0.239983,59.515880
DIESEL,124,0.030824,3.822205


# 🔄 Correção de Inconsistências de Preços

## 📊 Estratégia de Correção

### Para Vendas com Prejuízo (Venda < Compra):
- **Método**: Ajuste com margem de segurança
- **Fórmula**:  
  `Novo Valor de Venda = Valor de Compra × (1 + margem_seguranca)`

- **Vantagens**:
  - Elimina prejuízos nas vendas
  - Mantém margem de lucro mínima configurável
  - Aplica correção apenas onde necessário

### Para Vendas Válidas (Venda ≥ Compra):
- **Método**: Mantém os valores originais
- **Vantagens**:
  - Preserva os dados corretos
  - Não altera registros que já estão consistentes
  - Mantém a integridade das informações

## ⚙️ Configuração Padrão
- **Margem de Segurança**: 10% (0.10)
- **Personalização**: Ajustável conforme necessidade
- **Não-destrutivo**: Nunca altera o DataFrame original


In [39]:
def corrigir_inconsistencias(df, margem_seguranca=0.10, nome_conjunto="Dados"):
    """
    Corrige automaticamente as inconsistências nos preços,
    garantindo que valor_venda seja sempre maior que valor_compra.
    
    Parâmetros:
    - df: DataFrame com os dados
    - margem_seguranca: percentual a ser adicionado ao valor de compra (padrão: 10%)
    - nome_conjunto: Nome do conjunto de dados para referência
    
    Retorna:
    - DataFrame corrigido
    """
    # Verificar se as colunas existem
    colunas_necessarias = ['valor_venda', 'valor_compra']
    for col in colunas_necessarias:
        if col not in df.columns:
            raise KeyError(f"Coluna '{col}' não encontrada no DataFrame. Colunas disponíveis: {df.columns.tolist()}")
    
    # Identificar linhas com problemas
    mask = df['valor_venda'] < df['valor_compra']
    total_corrigir = mask.sum()
    
    if total_corrigir > 0:
        print(f"\n🔧 CORRIGINDO {total_corrigir:,} REGISTROS - {nome_conjunto.upper()}")
        print("="*60)
        
        # Aplicar correção
        df_corrigido = df.copy()
        df_corrigido.loc[mask, 'valor_venda'] = (
            df_corrigido.loc[mask, 'valor_compra'] * (1 + margem_seguranca)
        ).round(4)  # Arredonda para 4 casas decimais
        
        print(f"✅ Correção aplicada com sucesso! (Margem: {margem_seguranca*100:.0f}%)")
        print("="*60)
        return df_corrigido
    else:
        print(f"\n✅ Nenhuma correção necessária em {nome_conjunto}!")
        return df.copy()

# =============================================
# EXECUÇÃO DAS CORREÇÕES
# =============================================
print("\n" + "="*60)
print("🔄 INICIANDO PROCESSO DE CORREÇÃO".center(60))
print("="*60)

# Verificar colunas disponíveis antes de executar
print("\n📋 Colunas disponíveis no df_1s:", df_1s.columns.tolist())
print("📋 Colunas disponíveis no df_2s:", df_2s.columns.tolist())

# Primeiro Semestre
print("\n" + "="*30 + " PRIMEIRO SEMESTRE " + "="*30)
df_1s_corrigido = corrigir_inconsistencias(df_1s, nome_conjunto="Primeiro Semestre")

# Segundo Semestre
print("\n" + "="*30 + " SEGUNDO SEMESTRE " + "="*31)
df_2s_corrigido = corrigir_inconsistencias(df_2s, nome_conjunto="Segundo Semestre")

# Juntar os dados corrigidos
df_total_corrigido = pd.concat([df_1s_corrigido, df_2s_corrigido])

print("\n" + "="*60)
print("✅ CORREÇÕES CONCLUÍDAS COM SUCESSO!".center(60))
print("="*60)

# Verificação final
print("\n📊 RESUMO DAS CORREÇÕES:")
print(f"- Total de registros processados: {len(df_total_corrigido):,}")
print(f"- Registros no 1º semestre: {len(df_1s_corrigido):,}")
print(f"- Registros no 2º semestre: {len(df_2s_corrigido):,}")


              🔄 INICIANDO PROCESSO DE CORREÇÃO              

📋 Colunas disponíveis no df_1s: ['regiao_sigla', 'estado_sigla', 'municipio', 'revenda', 'cnpj_revenda', 'produto', 'data_coleta', 'valor_venda', 'valor_compra', 'unidade_medida', 'bandeira']
📋 Colunas disponíveis no df_2s: ['regiao_sigla', 'estado_sigla', 'municipio', 'revenda', 'cnpj_revenda', 'produto', 'data_coleta', 'valor_venda', 'valor_compra', 'unidade_medida', 'bandeira']

============================== PRIMEIRO SEMESTRE ==============================

🔧 CORRIGINDO 1,586 REGISTROS - PRIMEIRO SEMESTRE
✅ Correção aplicada com sucesso! (Margem: 10%)

============================== SEGUNDO SEMESTRE ===============================

🔧 CORRIGINDO 10,123 REGISTROS - SEGUNDO SEMESTRE
✅ Correção aplicada com sucesso! (Margem: 10%)

            ✅ CORREÇÕES CONCLUÍDAS COM SUCESSO!             

📊 RESUMO DAS CORREÇÕES:
- Total de registros processados: 1,196,848
- Registros no 1º semestre: 281,531
- Registros no 2º semestre: 91

In [40]:
df_2s.head()

,regiao_sigla,estado_sigla,municipio,revenda,cnpj_revenda,produto,data_coleta,valor_venda,valor_compra,unidade_medida,bandeira
0,CO,MS,CAMPO GRANDE,AUTO POSTO APARECIDA DO NORTE LTDA,86.807.609/0001-92,GASOLINA,2004-07-01,2.71,2.0888,R$ / litro,LIQUIGÁS
1,CO,MS,CAMPO GRANDE,AUTO POSTO APARECIDA DO NORTE LTDA,86.807.609/0001-92,ETANOL,2004-07-01,1.71,1.1877,R$ / litro,LIQUIGÁS
2,CO,MS,CAMPO GRANDE,AUTO POSTO APARECIDA DO NORTE LTDA,86.807.609/0001-92,DIESEL,2004-07-01,1.88,1.5279,R$ / litro,LIQUIGÁS
3,CO,MT,ALTA FLORESTA,AUTO POSTO BURIOL LTDA,04.481.758/0001-40,GASOLINA,2004-07-01,2.69,2.0735,R$ / litro,CBPI
4,CO,MT,ALTA FLORESTA,AUTO POSTO BURIOL LTDA,04.481.758/0001-40,ETANOL,2004-07-01,1.69,1.3080,R$ / litro,CBPI
